In [1]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from torch.cuda.amp import autocast, GradScaler
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoConfig, AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob
import random
import warnings
import gc
import os
import seaborn as sns
from tqdm.auto import tqdm
import torch.nn as nn
import torch
plt.style.use('seaborn-pastel')
sns.set_palette("winter_r")
warnings.filterwarnings('ignore')
tqdm.pandas()

/home/abe/anaconda3/envs/sc/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/abe/anaconda3/envs/sc/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Config:
    script = "roberta/feature_extraction"

    n_splits = 5
    seed = 42

    batch_size = 16
    n_classes = 4
    n_epochs = 10

    # bert
    model_name = "roberta-base"
    weight_decay = 2e-5
    beta = (0.9, 0.98)
    max_len = 128
    lr = 2e-5
    num_warmup_steps_rate = 0.01
    clip_grad_norm = None
    gradient_accumulation_steps = 1
    num_eval = 1

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Reka Env
    dir_path = "/home/abe/kaggle/signate-sc2022"


In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def path_setup(cfg):
    cfg.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    cfg.INPUT = os.path.join(Config.dir_path, 'input')
    cfg.OUTPUT = os.path.join(Config.dir_path, 'output')
    cfg.SUBMISSION = os.path.join(Config.dir_path, 'submissions')
    cfg.OUTPUT_EXP = os.path.join(cfg.OUTPUT, Config.script)
    cfg.EXP_MODEL = os.path.join(cfg.OUTPUT_EXP, "model")
    cfg.EXP_PREDS = os.path.join(cfg.OUTPUT_EXP, "preds")
    cfg.EXP_FIG = os.path.join(cfg.OUTPUT_EXP, "fig")
    cfg.NOTEBOOK = os.path.join(Config.dir_path, "Notebooks")
    cfg.SCRIPT = os.path.join(Config.dir_path, "scripts")

    # make dir
    for dir in [
            cfg.INPUT,
            cfg.OUTPUT,
            cfg.SUBMISSION,
            cfg.OUTPUT_EXP,
            cfg.EXP_MODEL,
            cfg.EXP_PREDS,
            cfg.EXP_FIG,
            cfg.NOTEBOOK,
            cfg.SCRIPT]:
        os.makedirs(dir, exist_ok=True)

    return cfg

In [4]:
seed_everything(Config.seed)
cfg = path_setup(Config)

In [9]:
class BERTModel(nn.Module):
    def __init__(self, model_name="roberta-base", criterion=None):
        super().__init__()
        self.criterion = criterion
        self.config = AutoConfig.from_pretrained(
            model_name,
            output_hidden_states=True
        )
        self.backbone = AutoModel.from_pretrained(
            model_name,
            config=self.config
        )
        self.fc = nn.Sequential(
            nn.Linear(self.config.hidden_size, 4),
        )

    def forward(self, inputs, labels=None):
        outputs = self.backbone(**inputs)["last_hidden_state"]
        outputs = outputs[:, 0, :]
        if labels is not None:
            logits = self.fc(outputs)
            loss = self.criterion(logits, labels)
            return logits, loss
        else:
            logits = self.fc(outputs)
            return logits


class BertSequenceVectorizer:
    def __init__(self, model, tokenizer, max_len):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = tokenizer
        self.bert_model = model
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = max_len

    def vectorize(self, sentence: str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor(
            [inputs], dtype=torch.long).to(
            self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():
            return seq_out.cpu().detach().numpy()
        else:
            return seq_out[0][0].detach().numpy()


def vectorize(df: pd.DataFrame, tokenizer, model_paths):
    assert "description" in df.columns
    df['feature'] = 0
    print('\n'.join(model_paths))
    for _, model_weight in enumerate(model_paths):
        model = BERTModel()
        model.load_state_dict(torch.load(model_weight))
        model = model.to(cfg.device)

        BSV = BertSequenceVectorizer(model, tokenizer, cfg.max_len)
        def _vectorize(x):
            print(x)
            x = BSV.vectorize(x)
            print(x)
            return BSV.vectorize(x)
        df['feature'] += df['description'].progress_apply(lambda x: _vectorize(x))

        del model
        gc.collect()

    df['feature'] = df['feature'] / len(model_paths)
    return pd.DataFrame(np.stack(df['feature']))


In [6]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)

In [7]:
model_paths = [p for p in sorted(glob(os.path.join(cfg.dir_path + "/output/roberta/baseline/model/", "fold*.pth")))]
train = pd.read_csv(os.path.join(cfg.INPUT, "train_cleaned.csv"))
print(train['description'].head(10))

0    develop cutting edge web applications perform ...
1    designs develops high quality scalable efficie...
2    functions point person network strategy work r...
3    work technical design development release depl...
4    quantify resources required task project relat...
5    participates standard business technical infor...
6    create project plans establish timelines estab...
7    facilitate pre sales initiatives live demonstr...
8    consolidate dashboards across team help drive ...
9    maintain improve existing predictive models ev...
Name: description, dtype: object


In [10]:
feat_train = vectorize(train, tokenizer, model_paths)
print(feat_train.head())
print(feat_train.shape)

/home/abe/kaggle/signate-sc2022/output/roberta/baseline/model/fold0.pth
/home/abe/kaggle/signate-sc2022/output/roberta/baseline/model/fold1.pth
/home/abe/kaggle/signate-sc2022/output/roberta/baseline/model/fold2.pth
/home/abe/kaggle/signate-sc2022/output/roberta/baseline/model/fold3.pth
/home/abe/kaggle/signate-sc2022/output/roberta/baseline/model/fold4.pth


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 1/1516 [00:00<00:05, 290.24it/s]

develop cutting edge web applications perform superbly across platforms work highly collaborative environment cross functional teams projects ranging weeks months length maintain high standard quality creatively strategically problem solve throughout product delivery process able effectively communicate work technical non technical peersbe excited new web technologies techniquesbuild solid front end architectures integrate easily systems technologiesworking closely disciplines back end ux design qa superior attention detail strong ability q one\ work required including cross browser cross platform displays performance


TypeError: RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dro